In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt


In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
!pip install ultralytics

In [4]:
from ultralytics import YOLO
import os
from IPython.display import display,Image
from IPython import display
display.clear_output()
!yolo mode=checks

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 532, in entrypoint
    raise ValueError(f"Invalid 'mode={mode}'. Valid modes are {MODES}.\n{CLI_HELP_MSG}")
ValueError: Invalid 'mode=<module 'ultralytics.utils.checks' from '/usr/local/lib/python3.10/dist-packages/ultralytics/utils/checks.py'>'. Valid modes are {'benchmark', 'train', 'val', 'track', 'predict', 'export'}.

    Arguments received: ['yolo', 'mode=checks']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of {'obb', 'classify', 'pose', 'segment', 'detect'}
                MODE (required) is one of {'benchmark', 'train', 'val', 'track', 'predict', 'export'}
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at ht

In [5]:
ls

drive/  sample_data/


In [6]:
cd

/root


In [7]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
os.chdir('/content/drive/My Drive/projects/yolov8_aerial_maritime')

In [9]:
ls

data/  data.yaml  README.dataset.txt  README.roboflow.txt


In [10]:
from google.colab import files


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
train_path_img='/content/drive/MyDrive/projects/yolov8_aerial_maritime/data/train/images'
train_path_label='/content/drive/MyDrive/projects/yolov8_aerial_maritime/data/train/labels'
val_path_img='/content/drive/MyDrive/projects/yolov8_aerial_maritime/data/valid/images'
val_path_label='/content/drive/MyDrive/projects/yolov8_aerial_maritime/data/valid/labels'
test_path_img='/content/drive/MyDrive/projects/yolov8_aerial_maritime/data/test/images'
test_path_label='/content/drive/MyDrive/projects/yolov8_aerial_maritime/data/test/labels'

In [25]:
cd data

[Errno 2] No such file or directory: 'data'
/content/drive/My Drive/projects/yolov8_aerial_maritime/data


In [26]:
import os
import shutil
import random
from tqdm.notebook import tqdm


In [27]:


def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/projects/yolov8_aerial_maritime/data') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images


------ PROCESS STARTED -------
--- This folder has a total number of 4 images---


  0%|          | 0/4 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/projects/yolov8_aerial_maritime/datav.jpg'

In [ ]:
from ultralytics import YOLO

# Load a YOLOv8 model (YOLOv8n, YOLOv8s, YOLOv8m, YOLOv8l, YOLOv8x)
model = YOLO('yolov8n.pt')  # You can choose a different model variant if needed

# Train the model
model.train(data='/content/drive/MyDrive/projects/yolov8_aerial_maritime/data.yaml', epochs=100, imgsz=640, batch=8)


Ultralytics YOLOv8.2.48 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/projects/yolov8_aerial_maritime/data.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, sho

100%|██████████| 755k/755k [00:00<00:00, 107MB/s]


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/drive/MyDrive/projects/yolov8_aerial_maritime/data/train/labels... 230 images, 0 backgrounds, 0 corrupt: 100%|██████████| 230/230 [01:09<00:00,  3.30it/s]

train: New cache created: /content/drive/MyDrive/projects/yolov8_aerial_maritime/data/train/labels.cache


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      3/100         0G      1.821      2.621      1.756         35        640: 100%|██████████| 29/29 [02:55<00:00,  6.04s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:13<00:00,  3.41s/it]

                   all         61        150      0.748      0.184      0.188       0.08



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      4/100         0G      1.849      2.597      1.871         59        640:  31%|███       | 9/29 [00:53<01:56,  5.83s/it]

''

In [31]:
import os

# Verify train images
print(os.listdir('/content/drive/MyDrive/projects/yolov8_aerial_maritime/data/train/images'))

# Verify val images
print(os.listdir('/content/drive/MyDrive/projects/yolov8_aerial_maritime/data/valid/images'))

# Verify test images
print(os.listdir('/content/drive/MyDrive/projects/yolov8_aerial_maritime/data/test/images'))


['DJI_0324_JPG.rf.2ddd1699cea446f7d3ba3ebd24821d1d.jpg', 'DJI_0324_JPG.rf.26e4b59d10b56abac113d1f534dbcb48.jpg', 'DJI_0331_JPG.rf.f883a7944c15b7c7e207d70f87e43e37.jpg', 'DJI_0324_JPG.rf.58cbcb20b3d6259ed9970b3f2f0cbff3.jpg', 'DJI_0261_JPG.rf.01b8557cc35f814b020b1154a97e57e4.jpg', 'DJI_0324_JPG.rf.223e12c0a7e10fe5f37ce2e63bcc6a2c.jpg', 'DJI_0315_JPG.rf.06b5176f8601463d99a33d9caab6ed31.jpg', 'DJI_0315_JPG.rf.423ecf64f62672c7900f7a47ed0e7220.jpg', 'DJI_0306_JPG.rf.0de9c5b63f665e1e9bca000f825140ea.jpg', 'DJI_0309_JPG.rf.2ba0998a678544ae9646fd5b28d96717.jpg', 'DJI_0298_JPG.rf.54a70b0c7b7b92f4cf1e39dfc73f2aa1.jpg', 'DJI_0324_JPG.rf.4cd984dd98fa0952ef3e70c996835c28.jpg', 'DJI_0314_JPG.rf.f946db1435638de7b917aa4eb28f031a.jpg', 'DJI_0307_JPG.rf.55dab2883d8193a078b5e0e5707e8159.jpg', 'DJI_0309_JPG.rf.ff396b745fb670850426e9350cd57139.jpg', 'DJI_0264_JPG.rf.51f8dc1e35f7415164eaab1790db60bf.jpg', 'DJI_0261_JPG.rf.59b70123c75e6578da7f9af8e540f953.jpg', 'DJI_0264_JPG.rf.6073ce58664a3abec8f08958fdbd84